# Generate Reports
**Filename:** GenerateReports.ipynb <br>
**Author:** Laura Kaufmann <br>
**Purpose:** To automate the generation of essential reports prior to departure, including Safety Cards, the Itinerary for Emergency Contacts and daily itinerary elements<br>
**Methods:**<br>
- Create data tables and feature classes formatted in ways that support the map series driven SafetyCard_ layouts <br>
- Truncate and append the newly calculated data to the existing feature classes in the Reporting database which are already connected to the SafetyCard_ layouts <br>
- Script the export of the SafetyCard_ layouts to PDF formats <br>
- Script the merging of the exported PDFs to support printing <br>

**Resources:**<br>
- [ESRI's Introduction to Spatially Enabled Dataframes](https://developers.arcgis.com/python/guide/part1-introduction-to-sedf/)
- [Community question about generating a table from a dataframe](https://community.esri.com/t5/arcgis-pro-questions/convert-data-frame-to-stand-alone-table/td-p/1091036)
- [Community question about an error generating a table from a dataframe](https://community.esri.com/t5/python-questions/arcpy-da-numpyarraytotable-returns-quot/m-p/1027773/highlight/true#M60021)
- [ESRI's Documentation on Dynamic Text](https://pro.arcgis.com/en/pro-app/3.1/help/layouts/add-dynamic-text.htm)

|Date|Editor|Changes|
|---|---|---|
|11/11/2023|L. Kaufmann|File created|
|11/20/2023|L. Kaufmann|Updates Regions and GatheringPlaces in Reporting|
||||

## Packages and Functions

In [2]:
# Import packages
import logging
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.geometry import Geometry

import pandas as pd
from pandas import Series, DataFrame
import numpy as np

import os
import pyodbc
import sqlalchemy

# GEODATABASES
sr = arcpy.SpatialReference(3857)
arcpy.env.overwriteOutput = True

fldr = r"C:\Users\Laura\Documents\Keepsakes\Travel\TravelMaster"
scratch = os.path.join(fldr, r"Scratch.gdb")
reporting = os.path.join(fldr, r"Reporting.gdb")

# SOURCE FEATURE CLASSES
tripsFC = r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_DEV.sde\Travel_DEV.DBO.Trips"
tripFields = ['Trip_ID', 'TripStage']

regionsFC = r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_DEV.sde\Travel_DEV.DBO.Regions"

locationsFC = r"C:\Users\Laura\AppData\Roaming\Esri\ArcGISPro\Favorites\Travel_DEV.sde\Travel_DEV.DBO.Locations"

# SQL RELATIONAL (via sqlalchemy)
sql_conn = "mssql+pyodbc://DESKTOP-MICQHNR\SQLEXPRESS/Travel_DEV?trusted_connection=yes&driver=SQL+Server"
engine = sqlalchemy.create_engine(sql_conn)

logging.info('Packages imported; ready to begin')

2023-11-21 16:14:39,535 - NumExpr defaulting to 8 threads.
2023-11-21 16:14:44,025 - Packages imported; ready to begin


In [3]:
# Function to read SQL qry into a dataframe and write to the Reporting geodatabase
def updateReporting(fc, qry):
    logging.info('Beginning {}'.format(fc))

    # Define variables
    scratchTbl = os.path.join(scratch, fc)
    reportingTbl = os.path.join(reporting, fc)

    df = pd.read_sql(qry, con=engine) 
    df = df.fillna('')

    # Create the table in the scratch workspace
    dfArray = df.values
    npstruct = np.rec.fromrecords(dfArray)
    colnames = df.dtypes.index.tolist()
    npstruct.dtype.names = tuple(colnames)
    arcpy.da.NumPyArrayToTable(npstruct, scratchTbl)

    logging.info('SQL data read into a NumPy array')

    # Truncate and append in the reporting workspace
    arcpy.management.TruncateTable(in_table=reportingTbl)
    arcpy.management.Append(
        inputs=scratchTbl,
        target=reportingTbl,
        schema_type="TEST_AND_SKIP",
        field_mapping=None,
        subtype="",
        expression="",
        match_fields=None,
        update_geometry="NOT_UPDATE_GEOMETRY"
    )

    # Delete the scratch table
    arcpy.management.Delete(in_data=scratchTbl, data_type="")

    logging.info('{} updated in the Reporting geodatabase'.format(fc))

# Function to create the near table and join back participating featureclasses to retain attributes
def generateNear(locQry):
    inFC = locQry.spatial.to_featureclass(os.path.join(scratch, "inFC"))
    
    nearTable = arcpy.analysis.GenerateNearTable(
        in_features=regionsScratch,
        near_features=inFC,
        out_table=os.path.join(scratch, "NearTable"),
        search_radius=None,
        location="NO_LOCATION",
        angle="NO_ANGLE",
        closest="ALL",
        closest_count=len(locations.index),
        method="GEODESIC",
        distance_unit="Miles"
    )
    
    arcpy.management.JoinField(
        in_data=nearTable,
        in_field="NEAR_FID",
        join_table=inFC,
        join_field="OBJECTID",
        fields="loc_reg_id;location_id;loc_type;loc_name;loc_local_name;loc_street_address;loc_phone;loc_desc;loc_interest;loc_website;loc_img_link",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )
    
    arcpy.management.JoinField(
        in_data=nearTable,
        in_field="IN_FID",
        join_table=regionsScratch,
        join_field="OBJECTID",
        fields="region_id;reg_name;reg_parent",
        fm_option="NOT_USE_FM",
        field_mapping=None,
        index_join_fields="NO_INDEXES"
    )

# Function to append records to the GatheringPlaces table and delete intermediate data
def appendNear(nearTable, inFC):
    arcpy.management.Append(
        inputs=nearTable,
        target=rptGathering,
        schema_type="TEST_AND_SKIP",
        field_mapping=None,
        subtype="",
        expression="",
        match_fields=None,
        update_geometry="NOT_UPDATE_GEOMETRY"
    )
    
    arcpy.management.Delete(in_data=inFC, data_type="")
    arcpy.management.Delete(in_data=nearTable, data_type="")

## Create data and append to Reporting

### Contacts (from the Travelers module)

In [40]:
scContacts = r"SC_Contacts"

contactsTbl = """
SELECT dbo.TRAVELERS.FirstName AS Sort
	 , '<BOL>' + dbo.TRAVELERS.FirstName + ' ' + dbo.TRAVELERS.LastName + '</BOL>' as ContactName
	 , '<BOL>' + dbo.TRAVELERS.USPhone + '</BOL>' AS Phone
FROM dbo.TRIPS INNER JOIN
             dbo.TRAVELERS ON dbo.TRIPS.Trip_ID = dbo.TRAVELERS.Trip_ID
WHERE (dbo.TRIPS.TripStage = N'Actively Planning')

UNION

SELECT dbo.TRAVELERS.FirstName + CAST(dbo.TRAVELERS_CONTACTS.OBJECTID AS nvarchar) AS Sort
	 , '     ' + dbo.TRAVELERS_CONTACTS.EC_FirstName + ' ' + dbo.TRAVELERS_CONTACTS.EC_LastName AS ContactName
	 , dbo.TRAVELERS_CONTACTS.EC_Phone AS Phone
FROM   dbo.TRIPS INNER JOIN
            dbo.TRAVELERS ON dbo.TRIPS.Trip_ID = dbo.TRAVELERS.Trip_ID INNER JOIN
			dbo.TRAVELERS_CONTACTS ON dbo.TRAVELERS.Traveler_ID = dbo.TRAVELERS_CONTACTS.Traveler_ID
WHERE (dbo.TRIPS.TripStage = N'Actively Planning')
"""
updateReporting(scContacts, contactsTbl)

2023-11-11 17:40:06,664 - Beginning SC_Contacts
2023-11-11 17:40:06,804 - SQL data read into a NumPy array
2023-11-11 17:40:19,391 - SC_Contacts updated in the Reporting geodatabase


In [19]:
iecProfiles = r"IEC_Profiles"

profilesQry = """
SELECT dbo.TRAVELERS.Traveler_ID
     , dbo.TRAVELERS.FirstName + ' ' + dbo.TRAVELERS.LastName AS Name
     , dbo.TRAVELERS.Role
     , dbo.TRAVELERS.USPhone
     , dbo.TRAVELERS.Email
     , dbo.TRAVELERS.Traveler_ImgFile
     , STRING_AGG ( dbo.TRAVELERS_FACTS.Fact, '\n' )
FROM dbo.TRAVELERS 
    LEFT OUTER JOIN
        dbo.TRAVELERS_FACTS ON dbo.TRAVELERS.Traveler_ID = dbo.TRAVELERS_FACTS.Traveler_ID 
    INNER JOIN
        dbo.TRIPS ON dbo.TRAVELERS.Trip_ID = dbo.TRIPS.Trip_ID
WHERE (dbo.TRIPS.TripStage = N'Actively Planning')
GROUP BY dbo.TRAVELERS.Traveler_ID
     , dbo.TRAVELERS.FirstName
	 , dbo.TRAVELERS.LastName
     , dbo.TRAVELERS.Role
     , dbo.TRAVELERS.USPhone
     , dbo.TRAVELERS.Email
     , dbo.TRAVELERS.Traveler_ImgFile
"""

updateReporting(iecProfiles, profilesQry)

2023-11-12 16:25:26,051 - Beginning IEC_Profiles
2023-11-12 16:25:26,163 - SQL data read into a NumPy array
2023-11-12 16:25:31,670 - SC_Contacts updated in the Reporting geodatabase


In [20]:
contactsFC = r"IEC_Contacts"

contactsQry = """
SELECT dbo.TRAVELERS.Traveler_ID
     , dbo.TRAVELERS.FirstName + N' ' + dbo.TRAVELERS.LastName AS Traveler
     , dbo.TRAVELERS_CONTACTS.EC_FirstName + N' ' + dbo.TRAVELERS_CONTACTS.EC_LastName AS Contact
     , dbo.TRAVELERS_CONTACTS.EC_Relationship
     , dbo.TRAVELERS_CONTACTS.EC_Phone
FROM dbo.TRAVELERS 
        INNER JOIN
            dbo.TRIPS ON dbo.TRAVELERS.Trip_ID = dbo.TRIPS.Trip_ID 
        LEFT OUTER JOIN
            dbo.TRAVELERS_CONTACTS ON dbo.TRAVELERS.Traveler_ID = dbo.TRAVELERS_CONTACTS.Traveler_ID
WHERE (dbo.TRIPS.TripStage = N'Actively Planning')
"""
updateReporting(contactsFC, contactsQry)

2023-11-12 16:25:38,146 - Beginning IEC_Profiles
2023-11-12 16:25:38,263 - SQL data read into a NumPy array
2023-11-12 16:25:41,648 - SC_Contacts updated in the Reporting geodatabase


### Taxis (from the Regions module)

In [41]:
scTaxis = r"SC_Taxis"

augTaxis = """
SELECT dbo.REGIONS.Region_ID
     , dbo.REGIONS.Reg_Name
     , dbo.REGIONS_TAXIS.CompanyName
     , dbo.REGIONS_TAXIS.CompanyPhone
FROM dbo.TRIPS INNER JOIN
        dbo.REGIONS ON dbo.TRIPS.Trip_ID = dbo.REGIONS.Trip_ID INNER JOIN
        dbo.REGIONS_TAXIS ON dbo.REGIONS.Region_ID = dbo.REGIONS_TAXIS.Region_ID
WHERE (dbo.TRIPS.TripStage = N'Actively Planning') AND (dbo.REGIONS.Reg_Interest <> N'None')
"""

updateReporting(scTaxis, augTaxis)

2023-11-11 17:41:22,963 - Beginning SC_Taxis
2023-11-11 17:41:23,078 - SQL data read into a NumPy array
2023-11-11 17:41:29,169 - SC_Taxis updated in the Reporting geodatabase


### Regions

In [11]:
logging.info('Beginning Regions processing')

# Find the Trip_ID of the active trip
with arcpy.da.SearchCursor(tripsFC, tripFields) as cursor:
    for row in cursor:
        if row[1] == 'Actively Planning':
            trip = row[0]

# Filter the Region dataframe
regions = pd.DataFrame.spatial.from_featureclass(regionsFC)
regions = regions.loc[(regions['Trip_ID'] == trip) & (regions['Reg_Interest'] != 'None')]

# Join with itinerary dates
itineraryDates = """
SELECT dbo.LOCATIONS.Region_ID
     , MIN(dbo.ITINERARY.Start) AS Arrive
     , MAX(dbo.ITINERARY.End_) AS Depart
FROM  dbo.LOCATIONS 
     INNER JOIN
          dbo.ITINERARY ON dbo.LOCATIONS.Location_ID = dbo.ITINERARY.Reference_ID 
     INNER JOIN
          dbo.REGIONS ON dbo.LOCATIONS.Region_ID = dbo.REGIONS.Region_ID
WHERE (dbo.ITINERARY.Event_Type = N'Visit')
GROUP BY dbo.LOCATIONS.Region_ID
"""

dates = pd.read_sql(itineraryDates, con=engine)
dates['Region_ID'] = '{' + dates['Region_ID'] + '}'
regions = pd.merge(regions, dates, on=["Region_ID"])
regions = regions.sort_values(by=['Arrive'])

regions['regParent'] = np.where(regions['Reg_DayTrip'] == 'No', regions['Region_ID'], regions['Region_ID'].shift(1, axis=0))
regions

logging.info('Regions joined with itinerary dates')

# Join with Countries data
countriesQry = """
SELECT dbo.REGIONS_COUNTRIES.Country_ISO
     , dbo.REGIONS_COUNTRIES.Ctry_LocalName
     , dbo.REGIONS_COUNTRIES.Ctry_Name
     , dbo.REGIONS_COUNTRIES.Ctry_Currency
     , dbo.REGIONS_COUNTRIES.Ctry_ExchgRate
     , dbo.REGIONS_COUNTRIES.Ctry_Injury
     , dbo.REGIONS_COUNTRIES.Ctry_NonEmergency
     , dbo.REGIONS_COUNTRIES.Ctry_Emergency
     , dbo.REGIONS_COUNTRIES.Ctry_TaxiReg
     , dbo.REGIONS_COUNTRIES.Ctry_FlagFile
     , dbo.REGIONS_COUNTRIES.Ctry_PhoneCode
     , dbo.REGIONS_COUNTRIES.Ctry_StateDept
FROM dbo.REGIONS_COUNTRIES
"""

countries = pd.read_sql(countriesQry, con=engine)
regions = pd.merge(regions, countries, on=["Country_ISO"])
logging.info('Regions joined with countries')

regionsScratch = regions.spatial.to_featureclass(os.path.join(scratch, "Regions"))
logging.info('Regions exported to scratch for spatial analysis')

2023-11-21 17:02:19,802 - Regions joined with itinerary dates
2023-11-21 17:02:19,831 - Regions joined with countries
2023-11-21 17:03:21,125 - Regions exported to scratch for spatial analysis


### ! Gathering Places

In [13]:
# Truncate GatheringPlaces table in Reporting
rptGathering = os.path.join(reporting, r"GatheringPlaces")
arcpy.management.TruncateTable(in_table=rptGathering)

# List active Region_IDs
activeRegions = []
with arcpy.da.SearchCursor(regionsScratch, ['Region_ID']) as cursor:
    for row in cursor:
        activeRegions.append(row[0])

# Read and filter the locations featureclass
locations = pd.DataFrame.spatial.from_featureclass(locationsFC)
locations = locations[locations['Region_ID'].isin(activeRegions)]
locations = locations.loc[(locations['Loc_Interest'] != 'None')]
locations = locations.rename(columns = {'Region_ID' : 'Loc_Reg_ID'})

# Find nearest embassy
embassies = locations.loc[(locations['Loc_Type'] == 'Emergency')]
generateNear(embassies)
appendNear(os.path.join(scratch, "inFC"), os.path.join(scratch, "NearTable"))

In [141]:
# Find the hotel in the same region
hotels = locations.loc[(locations['Loc_Type'] == 'Hotel')]
generateNear(hotels)
arcpy.analysis.TableSelect(
    in_table="NearTable",
    out_table=os.path.join(scratch, "NearTable_Select"),
    where_clause="loc_reg_id = region_id"
)

# Find Region_IDs with no hotels
HotelRegions = []
with arcpy.da.SearchCursor(os.path.join(scratch, "NearTable_Select"), ['region_id']) as cursor:
    for row in cursor:
        HotelRegions.append(row[0])
        
noHotelRegions = []
with arcpy.da.SearchCursor(os.path.join(scratch, "NearTable"), ['region_id']) as cursor:
    for row in cursor:
        noHotelRegions.append(row[0])
        
needPlace = list(set(noHotelRegions) - set(HotelRegions))

# Append hotels and delete intermediate data
appendNear(os.path.join(scratch, "inFC"), os.path.join(scratch, "NearTable_Select"))
arcpy.management.Delete(in_data=os.path.join(scratch, "NearTable"), data_type="")

# If no hotel exists, find a transportation hub as the gathering place
gatheringType = ['Airport', 'Ferry Port', 'Inter-city Bus', 'Parking', 'Train']
gathering = locations[locations['Loc_Reg_ID'].isin(needPlace) & locations['Loc_Type'].isin(gatheringType)]

generateNear(gathering)
arcpy.analysis.TableSelect(
    in_table="NearTable",
    out_table=os.path.join(scratch, "NearTable_Select"),
    where_clause="loc_reg_id = region_id"
)

# Append gathering places and delete intermediate data
appendNear(os.path.join(scratch, "inFC"), os.path.join(scratch, "NearTable_Select"))
arcpy.management.Delete(in_data=os.path.join(scratch, "NearTable"), data_type="")

<Result 'true'>

### IEC_TopDailyAttractions

In [162]:
dailyAttractionsFC = "IEC_TopDailyAttractions"

attractionsQry = """
SELECT Region_ID
	 , Reg_Name
	 , Reg_LocalName
	 , StartDate
	 , STRING_AGG (LocName, '\n' ) AS Name
	 , MAX(CASE WHEN AttractionRank = 1 THEN Loc_ImgLink END) AS ImgLink
FROM
	(SELECT Region_ID
		 , Reg_Name
		 , Reg_LocalName
		 , StartDate
		 , LocName
		 , Loc_ImgLink
		 , row_number() over (partition by cast(StartDate as date) order by Interest desc) as AttractionRank
	FROM
		(SELECT DISTINCT dbo.REGIONS.Region_ID
			 , dbo.REGIONS.Reg_Name
			 , dbo.REGIONS.Reg_LocalName
			 , cast(dbo.ITINERARY.Start as date) as StartDate
			 , Loc_Name + COALESCE(' (' + Loc_LocalName + ')', '') AS LocName
			 , dbo.LOCATIONS.Loc_ImgLink
			 , ISNULL((SELECT SUM(SURVEYRESULTS.Score) / COUNT(SURVEYRESULTS.Traveler_ID) as 'InterestValue'
				   FROM [Travel_DEV].[dbo].[SURVEYRESULTS]
				   WHERE (SURVEYRESULTS.Feature_ID = LOCATIONS.Location_ID)
				   GROUP BY Feature_ID), (IIF(LOCATIONS.Loc_Interest = 'Low', 2, IIF(LOCATIONS.Loc_Interest = 'Medium', 5, IIF(LOCATIONS.Loc_Interest = 'High', 10, 1))))) AS Interest
		FROM dbo.ITINERARY 
				INNER JOIN
					dbo.LOCATIONS ON dbo.ITINERARY.Reference_ID = dbo.LOCATIONS.Location_ID 
				INNER JOIN
					dbo.TRIPS 
				INNER JOIN
					dbo.REGIONS ON dbo.TRIPS.Trip_ID = dbo.REGIONS.Trip_ID ON dbo.LOCATIONS.Region_ID = dbo.REGIONS.Region_ID
		WHERE (dbo.TRIPS.TripStage = N'Actively Planning') 
			  AND (dbo.REGIONS.Reg_Interest <> N'None') 
			  AND (dbo.ITINERARY.Event_Type <> N'Transit')
			  AND (dbo.LOCATIONS.Loc_Type IN ('Historic Site', 'Landmark', 'Market', 'Museum', 'Observation Area', 'Other', 'Religious Site'))) uniqueAttractions) rankedAttractions
WHERE (AttractionRank < 6)
GROUP BY Region_ID
	   , Reg_Name
	   , Reg_LocalName
	   , StartDate
"""

updateReporting(dailyAttractionsFC, attractionsQry)

2023-11-20 23:58:21,768 - Beginning IEC_TopDailyAttractions
2023-11-20 23:58:21,905 - SQL data read into a NumPy array
2023-11-20 23:58:25,650 - IEC_TopDailyAttractions updated in the Reporting geodatabase


In [ ]:
# Title page
## Dates of trip
# Table of contents
## Map
## Calendar
## Regions